In [1]:
from fpl import Loader, Player, Team, SimpleExpectedPointsCalculator, Optimizer
import heapq

## Dowloading your personal FPL team

To download your own personal FPL team use `Loader.get_my_team(login, password, manager_id, how, filename)`

To do download from the API you need your login details and `manager_id`. To get your `manager_id`:
1. Log on to [Fantasy Premier League](https://fantasy.premierleague.com/) and navigate to the points tab.
2. Your `manager_id` should appear somewhere in the URL. [FPL APIs Explained](https://www.oliverlooney.com/blogs/FPL-APIs-Explained#how-to-use-authenticated-endpoints) has some screenshots of this.

If the download from the API fails, you can manually download the json blob as a local file:
1. Log on to [Fantasy Premier League](https://fantasy.premierleague.com/)
2. View a json blob of your team by typing this URL `https://fantasy.premierleague.com/api/my-team/{manager_id}/` into your browser with the appropriate `manager_id`
3. Save the blob locally for example as `my_team.json`


In [2]:
# team = Loader.get_my_team(login="email", password="password", manager_id=123456, how="api")
team = Loader.get_my_team(
    login="", password="", manager_id=0, how="local", filename="resources/my_team.json"
)
print(team)

---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
Player(element=521, name='Fabianski', position=1, club=19, cost=41)
---------------------------------------------------------------------------
{DEFS}
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
Player(element=475, name='Taylor', position=2, club=17, cost=39)
---------------------------------------------------------------------------
{MIDS}
Player(element=99, name='Mbeumo', position=3, club=4, cost=78)
Player(element=182, name='Palmer', position=3, club=6, cost=109)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=366, name='B.Fernandes', position=3, club=14, cost=84)
Player(element=585

## Creating a candidate list

Create a `candidates` list of players you want to transfer in, you can use the `Loader.find_matching_players(search_name, matching_threshold)` function below to help (remember to search by a player's name as it appears on the fpl website)

In [3]:
Loader.find_matching_players(search_name="alexander arnold", threshold=50)

[(Player(element=115, name='Baleba', position=3, club=5, cost=50),
  'Carlos Baleba'),
 (Player(element=223, name='Garner', position=3, club=8, cost=49),
  'James Garner'),
 (Player(element=240, name='Andreas', position=3, club=9, cost=50),
  'Andreas Hoelgebaum Pereira'),
 (Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=74),
  'Trent Alexander-Arnold'),
 (Player(element=642, name='André', position=3, club=20, cost=50),
  'André Trindade da Costa Neto')]

In [4]:
candidates = [
    Player(element=398, name="Gordon", position=3, club=15, cost=77),
    Player(element=339, name="Virgil", position=2, club=12, cost=64),
    Player(element=354, name="Kovačić", position=3, club=13, cost=54),
    Player(element=321, name="Gakpo", position=4, club=12, cost=76),
    Player(element=364, name="Amad", position=3, club=14, cost=56),
]

## Optimizing your team

`Optimizer.calc_optimial_formation()` is used to calculate the best possible for a particular gameweek.

`Optimizer.calc_discounted_reward_player()` is used to calculate how you think each player will perform on an individual basis over the coming gameweeks.

`Optimizer.calc_discounted_reward_team()` is used to calculate how you think a particular team will perform over the coming gameweeks. Each week the formation will be optimized to play the best possible team.

`Optimizer.calc_optimal_team()` is used to calculate the top three possible teams given a list of candidates.

Parameters:
- `:param epc:` Your prediction how a particular player will score on a particular gameweek is summarized by an `ExpectedPointsCalculator`. Here we use a `SimpleExpectedPointsCalculator` but we show below how to make your own expected points calculator below.

In [5]:
{
    c: Optimizer.calc_discounted_reward_player(
        player=c,
        epc=SimpleExpectedPointsCalculator,
        gameweek=Loader.get_next_gameweek(),
        horizon=5,
        gamma=1,
    )
    for c in candidates
}

{Player(element=398, name='Gordon', position=3, club=15, cost=77): 23.625,
 Player(element=339, name='Virgil', position=2, club=12, cost=64): 22.125,
 Player(element=354, name='Kovačić', position=3, club=13, cost=54): 19.875000000000004,
 Player(element=321, name='Gakpo', position=4, club=12, cost=76): 25.0,
 Player(element=364, name='Amad', position=3, club=14, cost=56): 30.625}

In [6]:
top_three = Optimizer.calc_optimal_teams(
    team,  # Your current fpl team.
    candidates,  # Whichever players you want to transfer in.
    SimpleExpectedPointsCalculator,  # How you want to calculate the expected points for each player.
    gameweek=Loader.get_next_gameweek(),  # Which gameweek do you want to start on.
    horizon=5,  # How many gameweeks do you want to calculate over.
    max_transfers=2,  # The maximum number of transfers allowed in the optimization.
    gamma=0.8,  # The discount factor.
    wildcard=False,  # Flag when True turns off the transfer adjustment.
)
third_best_score, third_best_team = heapq.heappop(top_three)
second_best_score, second_best_team = heapq.heappop(top_three)
first_best_score, first_best_team = heapq.heappop(top_three)
print("Score: {}".format(first_best_score))
print(first_best_team)

Score: 253.79481
---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
Player(element=521, name='Fabianski', position=1, club=19, cost=41)
---------------------------------------------------------------------------
{DEFS}
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
Player(element=475, name='Taylor', position=2, club=17, cost=39)
---------------------------------------------------------------------------
{MIDS}
Player(element=99, name='Mbeumo', position=3, club=4, cost=78)
Player(element=182, name='Palmer', position=3, club=6, cost=109)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=364, name='Amad', position=3, club=14, cost=56)
Player(e

In [10]:
Optimizer.calc_optimal_formation(
    first_best_team, SimpleExpectedPointsCalculator, Loader.get_next_gameweek()
)

Formation(total_exp_points=81.05624999999999, gkps=frozenset({Player(element=201, name='Henderson', position=1, club=7, cost=44)}), defs=frozenset({Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72), Player(element=70, name='Kerkez', position=2, club=3, cost=47), Player(element=231, name='Mykolenko', position=2, club=8, cost=43)}), mids=frozenset({Player(element=182, name='Palmer', position=3, club=6, cost=109), Player(element=364, name='Amad', position=3, club=14, cost=56), Player(element=99, name='Mbeumo', position=3, club=4, cost=78), Player(element=328, name='M.Salah', position=3, club=12, cost=132), Player(element=366, name='B.Fernandes', position=3, club=14, cost=84)}), fwds=frozenset({Player(element=447, name='Wood', position=4, club=16, cost=70), Player(element=401, name='Isak', position=4, club=15, cost=92)}), captain=Player(element=328, name='M.Salah', position=3, club=12, cost=132))


In [8]:
Optimizer.calc_optimal_formation(
    second_best_team, SimpleExpectedPointsCalculator, Loader.get_next_gameweek()
)

Formation(total_exp_points=80.85625, gkps=frozenset({Player(element=201, name='Henderson', position=1, club=7, cost=44)}), defs=frozenset({Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72), Player(element=70, name='Kerkez', position=2, club=3, cost=47), Player(element=231, name='Mykolenko', position=2, club=8, cost=43)}), mids=frozenset({Player(element=585, name='I.Sarr', position=3, club=7, cost=57), Player(element=182, name='Palmer', position=3, club=6, cost=109), Player(element=364, name='Amad', position=3, club=14, cost=56), Player(element=99, name='Mbeumo', position=3, club=4, cost=78), Player(element=328, name='M.Salah', position=3, club=12, cost=132)}), fwds=frozenset({Player(element=447, name='Wood', position=4, club=16, cost=70), Player(element=401, name='Isak', position=4, club=15, cost=92)}), captain=Player(element=328, name='M.Salah', position=3, club=12, cost=132))

In [9]:
Optimizer.calc_optimal_formation(
    team, SimpleExpectedPointsCalculator, Loader.get_next_gameweek()
)

Formation(total_exp_points=79.85625, gkps=frozenset({Player(element=201, name='Henderson', position=1, club=7, cost=44)}), defs=frozenset({Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72), Player(element=70, name='Kerkez', position=2, club=3, cost=47), Player(element=231, name='Mykolenko', position=2, club=8, cost=43)}), mids=frozenset({Player(element=585, name='I.Sarr', position=3, club=7, cost=57), Player(element=182, name='Palmer', position=3, club=6, cost=109), Player(element=99, name='Mbeumo', position=3, club=4, cost=78), Player(element=328, name='M.Salah', position=3, club=12, cost=132), Player(element=366, name='B.Fernandes', position=3, club=14, cost=84)}), fwds=frozenset({Player(element=447, name='Wood', position=4, club=16, cost=70), Player(element=401, name='Isak', position=4, club=15, cost=92)}), captain=Player(element=328, name='M.Salah', position=3, club=12, cost=132))